<a href="https://colab.research.google.com/github/krugis/alitolgaeskici/blob/main/crawl_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
## reference https://nariyoo.com/python-how-to-run-selenium-in-google-colab/
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,860 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,519 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubuntu jammy-

In [3]:
!pip install chromedriver-autoinstaller


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import chromedriver_autoinstaller

import sys

sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# Add arguments to disable images and unnecessary resources
chrome_options.add_argument('--disable-images')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
chrome_options.page_load_strategy = 'eager'  # Use 'eager' page load strategy

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)


# Function to get tweets from a specific user
def get_tweets(username, num_tweets=5):
    url = f"https://twitter.com/{username}"
    driver.get(url)
    time.sleep(2)  # Reduced initial wait time

    all_tweets = []  # Store all extracted tweets
    seen_tweets = set()  # Store tweet texts to avoid duplicates
    scroll_attempts = 0  # Track scroll attempts
    max_scroll_attempts = 10  # Maximum number of scroll attempts

    while len(all_tweets) < num_tweets and scroll_attempts < max_scroll_attempts:
        # Scroll down to load more tweets (before parsing)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)  # Wait for new tweets to load
        scroll_attempts += 1  # Increment scroll attempts

        # Parse the page source once outside the loop
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tweets = soup.find_all('article', {'data-testid': 'tweet'})

        # Process tweets using list comprehension and JavaScript execution once for all tweets
        new_tweets = [
            (
                tweet.find('div', {'data-testid': 'tweetText'}).text if tweet.find('div', {'data-testid': 'tweetText'}) else None,
                driver.execute_script(
                    """
                    const tweetHTML = arguments[0];
                    const tweet = document.createElement('div');
                    tweet.innerHTML = tweetHTML;

                    return {
                        comment_count: tweet.querySelector('[data-testid="reply"]')?.getAttribute('aria-label')?.split(' ')[0] || '0',
                        retweet_count: tweet.querySelector('[data-testid="retweet"]')?.getAttribute('aria-label')?.split(' ')[0] || '0',
                        like_count: tweet.querySelector('[data-testid="like"]')?.getAttribute('aria-label')?.split(' ')[0] || '0',
                    };
                    """,
                    str(tweet),  # Pass the tweet's outer HTML as a string
                )
            )
            for tweet in tweets
            if tweet.find('div', {'data-testid': 'tweetText'}) is not None and tweet.find('div', {'data-testid': 'tweetText'}).text not in seen_tweets
        ]

        # Extend the list of tweets and update the seen tweets set
        all_tweets.extend([(tweet_text, tweet_data['comment_count'], tweet_data['retweet_count'], tweet_data['like_count'])
                            for tweet_text, tweet_data in new_tweets])
        seen_tweets.update([tweet_text for tweet_text, _ in new_tweets])

    # Print or return the extracted tweets
    for tweet in all_tweets:
        print("Tweet:", tweet[0])
        print("Comments:", tweet[1])
        print("Retweets:", tweet[2])
        print("Likes:", tweet[3])
        print("-----")


# Example usage
get_tweets('GoatIndexAI', num_tweets=10)

# Close the WebDriver
driver.quit()

Tweet: Introducing Solana AI Ecosystem Airdrop by @GoatIndexAI

A movement to rally the Solana AI Ecosystem to grow 100x from here.

Launching on @MeteoraAG on Jan 16th.
Comments: 387
Retweets: 505
Likes: 1529
-----
Tweet:  BREAKING: GoatIndex Agent can now detect ANY new CAs posted by Trump family/cabal!

If you had bought $TRUMP the second its CA was posted on his X account, you could have made 30x your money.

You really think other Trump family or cabal members won’t drop their own CAs? 
Comments: 28
Retweets: 33
Likes: 182
-----
Tweet: After tonight, 50M new users are onboarding to @solana and @MeteoraAG.

@solana to $1T
@MeteoraAG to $100B
Comments: 5
Retweets: 7
Likes: 54
-----
Tweet: Good News: Our Twitter CA Monitoring tool (with image recognition capability) works for the Trump Tweet

Bad News: $TRUMP is not a Solana AI Project so we did not monitor his Twitter Profile
Comments: 5
Retweets: 6
Likes: 61
-----
